In [1]:
!pip install langfuse


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import sys
import os 
import nest_asyncio

# Sanity check
print(sys.executable)
nest_asyncio.apply()

os.environ["LANGFUSE_SECRET_KEY"]="sk-lf-305fb0a1-0d3f-4732-841a-baeed1abc7ff"
os.environ["LANGFUSE_PUBLIC_KEY"]="pk-lf-b044ea0d-bbbb-4e59-9a38-b814356fde31"
os.environ["LANGFUSE_HOST"]="https://cloud.langfuse.com"


os.environ["OPENAI_API_KEY"]="sk-"

#os.environ["AZURE_OPENAI_ENDPOINT"] = "https://oai-llmrag-training.openai.azure.com/"
#ENGINE_NAME = "gpt-4o-mini"



/Users/amorvan/Documents/code_dw/llm_collection/venv/bin/python3.13


In [18]:
import os
from pydantic import BaseModel, Field
from llama_index.core.workflow import (
    Workflow,
    step,
    Event,
    Context,
    StartEvent,
    StopEvent
)
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from langfuse.decorators import observe, langfuse_context

In [19]:
llm = OpenAI(temperature=0.05, model="gpt-4o-mini")


class EchoWorkflow(Workflow):
    """A dummy workflow with only one step sending back the input given."""

    @observe(capture_input=False)
    @step()
    async def run_step(self, ev: StartEvent) -> StopEvent:
        langfuse_context.update_current_trace(
            session_id="0000000",
            metadata={"project": "very_important"}
        )
        message = str(ev.get("message", ""))
        langfuse_context.update_current_observation(
            input=message, 
        )
        documents = SimpleDirectoryReader(
            input_files=["../notebooks/paul_graham_essay.txt"],
        ).load_data()
        splitter = SentenceSplitter(chunk_size=256)
        nodes = splitter.get_nodes_from_documents(documents)
        retriever_top_2 = BM25Retriever.from_defaults(
            nodes=nodes,
            similarity_top_k=2,
            stemmer=Stemmer.Stemmer("english"),
            language="english",
        )
        nodes = retriever_top_2.retrieve(message)
        context = "\n ".join([x.text for x in nodes])
        response = llm.complete(f"Summarize this : {context}")
        return StopEvent(result=response.text)

In [20]:
w = EchoWorkflow()
r = await w.run(message="computer")

In [21]:
r

"The advent of microcomputers revolutionized personal computing by allowing users to interact directly with a computer on their desk, rather than relying on punch cards. The narrator recalls the excitement of watching a friend build his own microcomputer from a Heathkit kit, which sparked feelings of envy. After years of persuasion, the narrator's father finally bought a TRS-80 in 1980, which marked the beginning of the narrator's programming journey. They created simple games, a model rocket flight predictor, and a word processor used by their father for writing. A memorable moment occurred when the narrator realized that programs could run indefinitely, highlighting the shift in computing dynamics."

## Get traces

In [42]:
traces = langfuse.fetch_traces()
traces.data[0]

TraceWithDetails(id='26c15e49-cb32-4884-a66b-8d697f148dbe', timestamp=datetime.datetime(2024, 12, 4, 20, 54, 42, 636000), name='run_step', input='computer', output='{"result":"The advent of microcomputers revolutionized personal computing by allowing users to interact directly with a computer on their desk, rather than relying on punch cards. The narrator recalls the excitement of watching a friend build his own microcomputer from a Heathkit kit, which sparked feelings of envy. After years of persuasion, the narrator\'s father finally bought a TRS-80 in 1980, which marked the beginning of the narrator\'s programming journey. They created simple games, a model rocket flight predictor, and a word processor used by their father for writing. A memorable moment occurred when the narrator realized that programs could run indefinitely, highlighting the shift in computing dynamics."}', session_id='0000000', release=None, version=None, user_id=None, metadata={'project': 'very_important'}, tags=

## Get a dataset

In [33]:
from langfuse import Langfuse

langfuse = Langfuse()
dataset = langfuse.get_dataset("MyDataset")

In [34]:
dataset

In [35]:
for item in dataset.items:
    if str(item.status) == "DatasetStatus.ACTIVE":
        print(item.input, item.expected_output)

{"args":[],"kwargs":{"ev":{"_data":{"message":"computer"}}}} {"result":"The advent of microcomputers revolutionized personal computing, allowing users to interact directly with computers on their desks rather than relying on punch cards. The author recalls feeling impressed and envious when a friend built his own microcomputer from a Heathkit kit. After years of persuasion, the author finally convinced his father to buy a TRS-80 in 1980, which marked the beginning of his programming journey. He created simple games, a model rocket prediction program, and a word processor used by his father for writing. A memorable moment for the author was realizing that programs could run indefinitely, highlighting the shift in computing capabilities."}


In [37]:
item.__dict__

{'id': 'cm4abe9to00h913yilynrk1v2',
 'status': <DatasetStatus.ARCHIVED: 'ARCHIVED'>,
 'input': '{"args":[],"kwargs":{"ev":{"_data":{"message":"Hello!"}}}}',
 'expected_output': '{"result":"Message received: [18] The worst thing about leaving YC was not working with Jessica anymore. We\'d been working on YC almost the whole time we\'d known each other, and we\'d neither tried nor wanted to separate it from our personal lives, so leaving was like pulling up a deeply rooted tree.\\n\\n[19] One way to get more precise about the concept of invented vs discovered is to talk about space aliens. Any sufficiently advanced alien civilization would certainly know about the Pythagorean theorem, for example. I believe, though with less certainty, that they would also know about the Lisp in McCarthy\'s 1960 paper.\\n\\nBut if so there\'s no reason to suppose that this is the limit of the language that might be known to them. Presumably aliens need numbers and errors and I/O too. So it seems likely t

## A/B testing 

In [16]:
prompt = langfuse.get_prompt("JokeTemplate1", version=1)
compiled_prompt = prompt.compile(topic="cats")

compiled_prompt

'Tell me a joke about cats'

In [43]:
prompt = langfuse.get_prompt("JokeTemplate1", version=2)
compiled_prompt = prompt.compile(topic="cats")

compiled_prompt

'Tell me a funny joke about cats'